In [ ]:
#https://www.christianhaller.me/blog/projectblog/2020-07-07-How-to-train-your-NLP-Chatbot/
#https://github.com/ChristianHallerX/DataScienceProjects/tree/master/Data/chatbot_data
#https://signal.onepointltd.com/post/102ge3r/training-a-question-answering-nlp-model
#https://medium.com/axinc-ai/convert-keras-models-to-tensorflow-lite-e654994fb93c

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
import json
import pickle

import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:


import pandas as pd

train_json = pd.read_json('/content/sample(1).json')

train_json
pd.set_option('display.max_colwidth', 300)
train_json.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])



,intents
0,"{'tags': 'program meanning', 'patterns': ['Meaning of Program and Course?', ' What does the course means?', 'What is a program?'], 'response': 'B.Com is a Program; Introduction to Accounting is a Course (Subject)'}"
1,"{'tags': 'course meanning', 'patterns': ['Meaning of Course?', 'What does the course means?', 'What is a program?'], 'response': 'B.Tech is a Program; Introduction to Finance is a Course (Subject)'}"
2,"{'tags': 'meanning course', 'patterns': ['Meaning of Course?', 'What does the course means?', 'What is a program?'], 'response': 'M.Tech is a Program; Introduction to Programming is a Course (Subject)'}"
3,"{'tags': 'meanning program', 'patterns': ['Meaning of Program?', 'What does the course means?', 'What is a program?'], 'response': 'Phd is a Program; Introduction to teaching is a Course (Subject)'}"
4,"{'tags': 'documents btech', 'patterns': ['What documents are required to be submitted for admission in B.tech?', ' Documents for B.Tech? ', 'B. Tech Documents Required? ', 'Required Documentation B.Tech?'], 'response': 'You need to submit color copies of Grade Sheet of 10th, 12th and adharcard/passport'}"
5,"{'tags': 'btech documents', 'patterns': ['What documents are required to be submitted for admission in B.tech?', ' Documents for B.Tech? ', 'B. Tech Documents Required? ', 'Required Documentation B.Tech?'], 'response': 'Grade Sheet of 10th, 12th and adharcard/passport (colour copies)'}"
6,"{'tags': 'document ug', 'patterns': ['What documents are required to be submitted for admission in UG?', ' Documents for UG? ', 'UG Documents Required? ', 'Required Documentation UG?'], 'response': 'Marksheet of class 10th, 12th and adharcard/passport (colour copies)'}"
7,"{'tags': 'ug document', 'patterns': ['What documents are required to be submitted for admission in UG?', ' Documents for UG? ', 'UG Documents Required? ', 'Required Documentation UG?'], 'response': 'Report Card of class 10th, 12th and adharcard/passport (colour copies)'}"
8,"{'tags': 'documents Mtech', 'patterns': ['What documents are required to be submitted for admission in Mtech?', ' Documents for Mtech? ', ' Mtech Documents Required? ', 'Required Documentation Mtech?'], 'response': 'You need to submit color copies Grade Sheet of 10th, 12th, UG consolidated marksheet with Degree/Provisional and adharcard/passport'}"
9,"{'tags': 'Mtech documents', 'patterns': ['What documents are required to be submitted for admission in Mtech?', ' Documents for Mtech? ', 'Mtech Documents Required? ', 'Required Documentation Mtech?'], 'response': 'Grade Sheet of 10th, 12th, UG consolidated marksheet with Degree/Provisional and adharcard/passport (colour copies)'}"


In [3]:
words = []                # all words in user input pattern
documents = []            # tokenized words and tags
classes = []              # tags
ignore_words = ['?', '!','(',')','//','&',',','.'] # also called stop words

data_file = open('/content/sample(1).json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize each word in user pattern and add to words list
        w = nltk.word_tokenize(str(pattern).lower().strip())
        words.extend(w)
        
        # add pattern token and tag to documents list
        
        documents.append((w, str(intent['tags']).lower().strip()))

        # add tag to classes list (only unique)
        if intent['tags'] not in classes:
            classes.append(str(intent['tags']).lower().strip())


In [ ]:
# classes
# words

['meaning',
 'of',
 'program',
 'and',
 'course',
 '?',
 'what',
 'does',
 'the',
 'course',
 'means',
 '?',
 'what',
 'is',
 'a',
 'program',
 '?',
 'meaning',
 'of',
 'course',
 '?',
 'what',
 'does',
 'the',
 'course',
 'means',
 '?',
 'what',
 'is',
 'a',
 'program',
 '?',
 'meaning',
 'of',
 'course',
 '?',
 'what',
 'does',
 'the',
 'course',
 'means',
 '?',
 'what',
 'is',
 'a',
 'program',
 '?',
 'meaning',
 'of',
 'program',
 '?',
 'what',
 'does',
 'the',
 'course',
 'means',
 '?',
 'what',
 'is',
 'a',
 'program',
 '?',
 'what',
 'documents',
 'are',
 'required',
 'to',
 'be',
 'submitted',
 'for',
 'admission',
 'in',
 'b.tech',
 '?',
 'documents',
 'for',
 'b.tech',
 '?',
 'b.',
 'tech',
 'documents',
 'required',
 '?',
 'required',
 'documentation',
 'b.tech',
 '?',
 'what',
 'documents',
 'are',
 'required',
 'to',
 'be',
 'submitted',
 'for',
 'admission',
 'in',
 'b.tech',
 '?',
 'documents',
 'for',
 'b.tech',
 '?',
 'b.',
 'tech',
 'documents',
 'required',
 '?',
 'r

In [4]:
lemmatizer = WordNetLemmatizer()

# lemmaztize and lower each item in words list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# convert to set to remove duplicates and back to list
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination of user input (patterns) and tags (intents)
print("{} documents {}".format(len(documents),documents))

# classes = unique intents
print("\n{} classes {}".format(len(classes),classes))

# words = all words, vocabulary
print("\n{} unique lemmatized words {}".format(len(words),words))

986 documents [(['meaning', 'of', 'program', 'and', 'course', '?'], 'program meanning'), (['what', 'does', 'the', 'course', 'means', '?'], 'program meanning'), (['what', 'is', 'a', 'program', '?'], 'program meanning'), (['meaning', 'of', 'course', '?'], 'course meanning'), (['what', 'does', 'the', 'course', 'means', '?'], 'course meanning'), (['what', 'is', 'a', 'program', '?'], 'course meanning'), (['meaning', 'of', 'course', '?'], 'meanning course'), (['what', 'does', 'the', 'course', 'means', '?'], 'meanning course'), (['what', 'is', 'a', 'program', '?'], 'meanning course'), (['meaning', 'of', 'program', '?'], 'meanning program'), (['what', 'does', 'the', 'course', 'means', '?'], 'meanning program'), (['what', 'is', 'a', 'program', '?'], 'meanning program'), (['what', 'documents', 'are', 'required', 'to', 'be', 'submitted', 'for', 'admission', 'in', 'b.tech', '?'], 'documents btech'), (['documents', 'for', 'b.tech', '?'], 'documents btech'), (['b.', 'tech', 'documents', 'required', 

In [ ]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
# create training list
training = []

# create empty array for output
output_empty = [0] * len(classes)
# print("output_empty::",output_empty)
# training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # print("Doc:::",doc)
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # print("pattern_words:::",pattern_words)
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # print("pattern_words:::",pattern_words)
    # create bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
   
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    # print(output_row )
    
    training.append([bag, output_row])
    
# shuffle features, make np.array
import random
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [8]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, y_train, y_test = train_test_split(train_x , train_y , test_size=0.33, random_state=42)
 

In [9]:

# X_train=np.expand_dims(X_train, axis=0)
# X_test=np.expand_dims(X_test, axis=0)
print(np.asarray(X_train).shape)
print(np.asarray(X_test).shape)
print(np.asarray(y_train).shape)
print(np.asarray(y_test).shape)

(660, 219)
(326, 219)
(660, 212)
(326, 212)


In [10]:
# model = tf.keras.Sequential()

# model.add(tf.keras.layers.Dense(256,input_shape=(len(train_x[0]),),  activation='relu'))
# model.add(tf.keras.layers.Dense(128, activation='relu'))
# model.add(tf.keras.layers.Dense(256, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(128,activation='relu'))
# model.add(tf.keras.layers.Dense(128,activation='relu'))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(32, activation='relu'))
#330/330 [==============================] - 1s 4ms/step - loss: 1.7471 - accuracy: 0.1003
# model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))



model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(256,input_shape=(len(train_x[0]),),  activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu'))



model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))

In [11]:
gd = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=gd,metrics=[tf.keras.metrics.Accuracy()])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               56320     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 256)               6

In [ ]:

# fit model
history = model.fit(np.array( X_train), np.array(y_train), epochs=1000, batch_size=1, verbose=1,validation_batch_size=(np.array( X_test), np.array(y_test)))

# save weights to file
model.save('chatbot_model.h5', history)

print("model created")

Epoch 1/1000
660/660 [==============================] - 6s 4ms/step - loss: 5.3481 - accuracy: 0.0000e+00
Epoch 2/1000
660/660 [==============================] - 3s 4ms/step - loss: 5.1049 - accuracy: 0.0000e+00
Epoch 3/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.9143 - accuracy: 0.0000e+00
Epoch 4/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.8584 - accuracy: 0.0000e+00
Epoch 5/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.7316 - accuracy: 0.0000e+00
Epoch 6/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.5907 - accuracy: 0.0000e+00
Epoch 7/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.5599 - accuracy: 5.7176e-05
Epoch 8/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.4780 - accuracy: 5.0029e-05
Epoch 9/1000
660/660 [==============================] - 3s 4ms/step - loss: 4.3095 - accuracy: 2.1441e-05
Epoch 10/1000
660/660 [=======================

In [ ]:
model.save('chatbot_model.h5', history)

In [ ]:
import tensorflow as tf


In [ ]:
model = tf.keras.models.load_model('/content/chatbot_model.h5')



OSError: ignored

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [ ]:
tflmodel = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp1tvwiq35/assets


In [ ]:
file = open( 'yourmodel.tflite' , 'wb' ) 


In [ ]:
file.write( tflmodel )

409588

In [ ]:
interpreter = tf.lite.Interpreter(model_path="/content/yourmodel.tflite")


In [ ]:
interpreter.allocate_tensors()


In [ ]:
input_details = interpreter.get_input_details()
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'serving_default_dense_6_input:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 183], dtype=int32),
  'shape_signature': array([ -1, 183], dtype=int32),
  'sparsity_parameters': {}}]

In [ ]:
output_details = interpreter.get_output_details()
output_details

[{'dtype': numpy.float32,
  'index': 19,
  'name': 'StatefulPartitionedCall:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 56], dtype=int32),
  'shape_signature': array([-1, 56], dtype=int32),
  'sparsity_parameters': {}}]